In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.hub import load_state_dict_from_url
import numpy as np

## Lets get the data, model and setup trainnig code

In [2]:
# train_loader = DataLoader(datasets.ImageNet("./", train=True, transform=transforms.ToTensor(), download=False), batch_size=128, shuffle=True)
# # test_loader = DataLoader(datasets.ImageNet("./", train=False, transform=transforms.ToTensor(), download=True), batch_size=128, shuffle=False)

# # print(f"Training images {len(train_loader.dataset)}, Test images {len(test_loader.dataset)}")

In [9]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)

model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
    'resnext50_32x4d': 'https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth',
    'resnext101_32x8d': 'https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth',
    'wide_resnet50_2': 'https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth',
    'wide_resnet101_2': 'https://download.pytorch.org/models/wide_resnet101_2-32ee1156.pth',
}

def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)
    
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x):
        return self._forward_impl(x)


def _resnet(arch, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model


def resnet18(pretrained=False, progress=True, **kwargs):
    r"""ResNet-18 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet18', BasicBlock, [2, 2, 2, 2], pretrained, progress,
                   **kwargs)

model_quan = resnet18()

Using cache found in C:\Users\67041/.cache\torch\hub\pytorch_vision_v0.10.0


## Extract weights

In [4]:
params = [(name, p.data.cpu().numpy()) for (name, p) in model.named_parameters()]

In [5]:
for (name, p) in params:
  print(f"Layer {name.split('.')[0]}, type {name.split('.')[1]}, shape {p.shape}")

Layer conv1, type weight, shape (64, 3, 7, 7)
Layer bn1, type weight, shape (64,)
Layer bn1, type bias, shape (64,)
Layer layer1, type 0, shape (64, 64, 3, 3)
Layer layer1, type 0, shape (64,)
Layer layer1, type 0, shape (64,)
Layer layer1, type 0, shape (64, 64, 3, 3)
Layer layer1, type 0, shape (64,)
Layer layer1, type 0, shape (64,)
Layer layer1, type 1, shape (64, 64, 3, 3)
Layer layer1, type 1, shape (64,)
Layer layer1, type 1, shape (64,)
Layer layer1, type 1, shape (64, 64, 3, 3)
Layer layer1, type 1, shape (64,)
Layer layer1, type 1, shape (64,)
Layer layer2, type 0, shape (128, 64, 3, 3)
Layer layer2, type 0, shape (128,)
Layer layer2, type 0, shape (128,)
Layer layer2, type 0, shape (128, 128, 3, 3)
Layer layer2, type 0, shape (128,)
Layer layer2, type 0, shape (128,)
Layer layer2, type 0, shape (128, 64, 1, 1)
Layer layer2, type 0, shape (128,)
Layer layer2, type 0, shape (128,)
Layer layer2, type 1, shape (128, 128, 3, 3)
Layer layer2, type 1, shape (128,)
Layer layer2, typ

In [9]:
#print(params)

## Visualize hidden activations

In [10]:
# print(model.children())
# out = list(model.children())[0](img.cuda()).data.cpu().numpy()

In [11]:
# import matplotlib.pyplot as plt
# %matplotlib inline

# for _ in range(out.shape[1]):
#   plt.figure(figsize=(1, 1))
#   plt.imshow(out[0, 0], cmap="gray")

In [12]:
for img, label in train_loader:
  break

In [13]:
print(label)

tensor([7, 0, 1, 4, 6, 3, 3, 1, 4, 7, 0, 4, 2, 8, 4, 5, 9, 0, 3, 1, 0, 5, 2, 2,
        2, 9, 8, 5, 8, 7, 3, 0, 5, 7, 0, 0, 1, 7, 1, 9, 1, 3, 0, 6, 0, 9, 6, 1,
        6, 4, 1, 3, 7, 2, 2, 7, 4, 4, 1, 1, 5, 4, 4, 4, 1, 1, 9, 7, 3, 4, 2, 4,
        4, 1, 2, 9, 7, 5, 6, 8, 3, 8, 9, 1, 1, 1, 1, 2, 6, 9, 7, 4, 7, 5, 3, 5,
        6, 4, 7, 0, 8, 6, 6, 3, 8, 4, 4, 9, 2, 8, 9, 0, 3, 6, 0, 4, 6, 3, 3, 7,
        2, 5, 1, 7, 9, 8, 3, 8])


In [16]:

path = "./ResNet/"
print(len(params[1]))
print(params[2][1].tolist())
# import os
# if not os.path.isdir(path):
#     os.mkdir(path)
# np.savetxt(fname=path+"label", delimiter=" ", X=label.tolist())
# print(get_acc(model, ([img,label],)))

2
[0.23071715235710144, 0.2538224756717682, -1.0542980817263015e-06, -0.66438889503479, -1.657055470616342e-08, 0.16152192652225494, 0.45450395345687866, -4.3019500139962474e-07, 0.30051374435424805, -8.005239578778856e-06, 0.3494180738925934, 0.3114806115627289, -0.24952976405620575, -3.474890036159195e-05, 0.1077263131737709, 0.2189706563949585, 0.3814125955104828, -0.5298821330070496, -0.6286441087722778, 0.5713980793952942, 0.29984691739082336, 0.5843037366867065, 0.4820215404033661, 0.3285263478755951, 0.1967170089483261, 0.1949618011713028, 0.1521454155445099, 0.08552236109972, 0.5131428241729736, 0.015236735343933105, 0.16644176840782166, 0.33239439129829407, 0.24921123683452606, 0.443366676568985, -0.280169278383255, -0.02038480155169964, -2.450687475175073e-07, 0.32134050130844116, -4.915174400821343e-08, 0.2377673089504242, 0.2329077273607254, 0.3152746260166168, 0.42776238918304443, 0.29312753677368164, 0.2637942433357239, 0.6759758591651917, 0.4291003942489624, 0.3456627428

In [21]:

# np.savetxt(fname=path+"input_0", delimiter=" ", X=img.cuda().view(-1, 784).tolist())
# np.savetxt(fname=path+"outputlayer1_0", delimiter=" ", X=model.output(img.cuda().view(128, 1, 28, 28))[0].data.cpu().view(-1))
# np.savetxt(fname=path+"outputlayer2_0", delimiter=" ", X=model.output(img.cuda().view(128, 1, 28, 28))[1].tolist())
# np.savetxt(fname=path+"outputlayer3_0", delimiter=" ", X=model.output(img.cuda().view(128, 1, 28, 28))[2].tolist())
# np.savetxt(fname=path+"outputlayer4_0", delimiter=" ", X=model.output(img.cuda().view(128, 1, 28, 28))[3].tolist())

# np.savetxt(fname=path+"weight1_0", delimiter=" ", X=params[0][1].reshape(5*5*1, 20).tolist())
# np.savetxt(fname=path+"bias1_0", delimiter=" ", X=params[1][1].tolist())
# np.savetxt(fname=path+"weight2_0", delimiter=" ", X=params[2][1].reshape(5*5*20, 50).tolist())
# np.savetxt(fname=path+"bias2_0", delimiter=" ", X=params[3][1].tolist())
# np.savetxt(fname=path+"weight3_0", delimiter=" ", X=params[4][1].tolist())
# np.savetxt(fname=path+"bias3_0", delimiter=" ", X=params[5][1].tolist())
# np.savetxt(fname=path+"weight4_0", delimiter=" ", X=params[6][1].tolist())
# np.savetxt(fname=path+"bias4_0", delimiter=" ", X=params[7][1].tolist())
for i in range(len(params)):
    print(params[i][1].shape)
    if len(params[i][1].shape)==4:
        np.savetxt(fname=path+"weight_"+str(i), delimiter=" ", X=params[i][1].reshape(params[i][1].shape[1]*\
            params[i][1].shape[2]*params[i][1].shape[3], params[i][1].shape[0]).tolist())
    else:\
        np.savetxt(fname=path+"weight_"+str(i), delimiter=" ", X=params[i][1].tolist())


(64, 3, 7, 7)
(64,)
(64,)
(64, 64, 3, 3)
(64,)
(64,)
(64, 64, 3, 3)
(64,)
(64,)
(64, 64, 3, 3)
(64,)
(64,)
(64, 64, 3, 3)
(64,)
(64,)
(128, 64, 3, 3)
(128,)
(128,)
(128, 128, 3, 3)
(128,)
(128,)
(128, 64, 1, 1)
(128,)
(128,)
(128, 128, 3, 3)
(128,)
(128,)
(128, 128, 3, 3)
(128,)
(128,)
(256, 128, 3, 3)
(256,)
(256,)
(256, 256, 3, 3)
(256,)
(256,)
(256, 128, 1, 1)
(256,)
(256,)
(256, 256, 3, 3)
(256,)
(256,)
(256, 256, 3, 3)
(256,)
(256,)
(512, 256, 3, 3)
(512,)
(512,)
(512, 512, 3, 3)
(512,)
(512,)
(512, 256, 1, 1)
(512,)
(512,)
(512, 512, 3, 3)
(512,)
(512,)
(512, 512, 3, 3)
(512,)
(512,)
(1000, 512)
(1000,)
